In [ ]:
!pip install -q pandas torch transformers accelerate scikit-learn joblib tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 62.1 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer, BertModel, RobertaTokenizerFast, RobertaForSequenceClassification
from tqdm import tqdm
import warnings
import joblib
import os

In [ ]:
sample_comment = '''Omg you played Psych!! I wrote the soundtrack for that game, I said to the developers that if there was one streamer I wanted to play Psych it would be Teo, how frigging cool!! Edit: At least you played some of it..😂'''

In [ ]:
GOOGLE_DRIVE_BASE_PATH = '/content/drive/MyDrive/ML_Models'
SENTIMENT_MODEL_PATH = os.path.join(GOOGLE_DRIVE_BASE_PATH, 'saved_model_sentiment')
EMOTION_MODEL_PATH = os.path.join(GOOGLE_DRIVE_BASE_PATH, 'saved_model_emotion')
CUSTOM_BERT_MODEL_PATH = '/content/drive/MyDrive/ML_Models/XGBOOST _Model/bert_model'
CUSTOM_BERT_TOKENIZER_PATH = '/content/drive/MyDrive/ML_Models/XGBOOST _Model/bert_tokenizer'
LIKE_COUNT_MODEL_PATH = '/content/drive/MyDrive/ML_Models/XGBOOST _Model/xgboost_BERT_embeddings.pkl'

SENTIMENT_LABELS = ['positive', 'negative', 'neutral']
EMOTION_LABELS = ["joy", "sadness", "anger", "fear", "disgust", "surprise", "neutral"]

try:
    from google.colab import drive
    drive.mount('/content/drive')
    print(f"Google Drive mounted. Looking for models in: {GOOGLE_DRIVE_BASE_PATH}")
except ImportError:
    print("Google Colab 'drive' module not found. Assuming not in Colab or Drive already mounted.")
except Exception as e:
    print(f"An error occurred during Google Drive mounting: {e}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive mounted. Looking for models in: /content/drive/MyDrive/ML_Models


In [ ]:
print(f"--- Loading custom BERT from '{CUSTOM_BERT_MODEL_PATH}' ---")
try:
    # Load your fine-tuned tokenizer and model from the specified paths
    embedding_tokenizer = BertTokenizer.from_pretrained(CUSTOM_BERT_TOKENIZER_PATH)
    embedding_model = BertModel.from_pretrained(CUSTOM_BERT_MODEL_PATH)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    embedding_model.to(device)
    print(f"✅ Custom BERT model and tokenizer loaded successfully on device: {device}")
except Exception as e:
    print(f"❌ Error loading custom BERT model/tokenizer: {e}")
    embedding_model = None

# --- Function to Generate Embeddings (No changes needed) ---
def get_single_bert_embedding(text):
    """Generates a BERT embedding for a single text string."""
    if embedding_model is None:
        return None
    embedding_model.eval()
    inputs = embedding_tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=128)
    inputs = {key: val.to(device) for key, val in inputs.items()}
    with torch.no_grad():
        outputs = embedding_model(**inputs)
    # Use the [CLS] token's embedding as the sentence representation
    cls_embedding = outputs.last_hidden_state[:, 0, :].cpu().numpy()
    return cls_embedding

--- Loading custom BERT from '/content/drive/MyDrive/ML_Models/XGBOOST _Model/bert_model' ---
✅ Custom BERT model and tokenizer loaded successfully on device: cpu


In [ ]:
def predict_on_text(text, model, tokenizer):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)

    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad(): #
        logits = model(**inputs).logits

    predicted_class_id = torch.argmax(logits, dim=-1).item()

    predicted_label = model.config.id2label[predicted_class_id]

    return predicted_label, predicted_class_id

In [ ]:
print("--- Loading Models and Tokenizers ---")

try:
    print(f"Loading sentiment model from: {SENTIMENT_MODEL_PATH}")
    sentiment_tokenizer = RobertaTokenizerFast.from_pretrained(SENTIMENT_MODEL_PATH)
    sentiment_model = RobertaForSequenceClassification.from_pretrained(SENTIMENT_MODEL_PATH)
    print("Sentiment model and tokenizer loaded successfully.")

    print(f"\nLoading emotion model from: {EMOTION_MODEL_PATH}")
    emotion_tokenizer = RobertaTokenizerFast.from_pretrained(EMOTION_MODEL_PATH)
    emotion_model = RobertaForSequenceClassification.from_pretrained(EMOTION_MODEL_PATH)
    print("Emotion model and tokenizer loaded successfully.")
except Exception as e:
    print(f"Error loading models or tokenizers: {e}")
    print("Ensure the paths are correct and the model files are intact.")
    exit()
try:
    predicted_sentiment, _ = predict_on_text(sample_comment, sentiment_model, sentiment_tokenizer)
except Exception as e:
    print(f"  Error predicting sentiment: {e}")

# Predict Emotion
try:
    predicted_emotion, _ = predict_on_text(sample_comment, emotion_model, emotion_tokenizer)
except Exception as e:
    print(f"  Error predicting emotion: {e}")

--- Loading Models and Tokenizers ---
Loading sentiment model from: /content/drive/MyDrive/ML_Models/saved_model_sentiment
Sentiment model and tokenizer loaded successfully.

Loading emotion model from: /content/drive/MyDrive/ML_Models/saved_model_emotion
Emotion model and tokenizer loaded successfully.


In [ ]:
def load_XGB_model(model_path):
    """Loads a saved XGBoost model from a .pkl file."""
    if not os.path.exists(model_path):
        print(f"❌ Error: Model file not found at {model_path}")
        return None
    try:
        model = joblib.load(model_path)
        print(f"✅ XGBoost model loaded successfully from {model_path}")
        return model
    except Exception as e:
        print(f"❌ Error loading XGBoost model: {e}")
        return None

like_count_model = load_XGB_model(LIKE_COUNT_MODEL_PATH)

✅ XGBoost model loaded successfully from /content/drive/MyDrive/ML_Models/XGBOOST _Model/xgboost_BERT_embeddings.pkl


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [13:50:24] WARNING: /workspace/src/gbm/gbtree.cc:363: 
  Loading from a raw memory buffer (like pickle in Python, RDS in R) on a CPU-only
  machine. Consider using `save_model/load_model` instead. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.  Changing `tree_method` to `hist`.
  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [13:50:24] WARNING: /workspace/src/gbm/gbtree.cc:388: Changing updater from `grow_gpu_hist` to `grow_quantile_histmaker`.
  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [13:50:24] WARNING: /workspace/src/context.cc:43: No visible GPU is found, setting device to CPU.
  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserW

In [ ]:
text_input_for_embedding = f"{sample_comment} [SEP] {predicted_emotion} [SEP] {predicted_sentiment}"
print(f"Step 3: Created combined input for embedding model.")
print(f"  > Combined Text: \"{text_input_for_embedding[:80]}...\"")

print("Step 4: Generating BERT embedding...")
final_embedding = get_single_bert_embedding(text_input_for_embedding)
if final_embedding is not None:
  print(f"  > Embedding generated with shape: {final_embedding.shape}")

Step 3: Created combined input for embedding model.
  > Combined Text: "Omg you played Psych!! I wrote the soundtrack for that game, I said to the devel..."
Step 4: Generating BERT embedding...
  > Embedding generated with shape: (1, 768)


In [ ]:
print("\n--- Predicting on Your Sample Comment ---")
print(f"\nSample Comment: \"{sample_comment}\"")
print(f"  --> Predicted Sentiment: {predicted_sentiment.upper()}")
print(f"  --> Predicted Emotion:   {predicted_emotion.capitalize()}")

if final_embedding is not None:

        predicted_likes = like_count_model.predict(final_embedding)
        final_prediction = max(0, predicted_likes[0])
        print(f"  --> Predicted Like Count: {round(final_prediction)}")
else:
    print("\nPrediction failed because one or more models could not be loaded.")


--- Predicting on Your Sample Comment ---

Sample Comment: "Omg you played Psych!! I wrote the soundtrack for that game, I said to the developers that if there was one streamer I wanted to play Psych it would be Teo, how frigging cool!! Edit: At least you played some of it..😂"
  --> Predicted Sentiment: POSITIVE
  --> Predicted Emotion:   Surprise
  --> Predicted Like Count: 154
